# Working with MongoDB/Mongoose
- You need to install [MongoDB](https://www.mongodb.com/) to use this notebook.

In [1]:
const mongoose = require('mongoose');

In [2]:
const dbName = "nodejs-learners-package-db";
const dbPort = '27017'; // Set your MongoDB port
const uri = `mongodb://127.0.0.1:${dbPort}/${dbName}`; // Replace with your actual connection uri

# Connecting to the database 

In [3]:
mongoose.connect(uri)
    .then(() => {
      console.log('Connected to MongoDB');
    }).catch((err) => {
      console.error('MongoDB connection error:', err);
    });

Promise { <pending> }

Connected to MongoDB


# Defining a schema and model

In [4]:
// Define a schema
/*
The { strict: true } option in Mongoose is used to enforce strict schema validation. 
When this option is enabled, Mongoose will prevent you from saving documents that contain fields
that are not defined in the schema.
*/
const customerSchema = new mongoose.Schema({
  name: String,
  email: { type: String, unique: true },
  age: Number,
  createdAt: { type: Date, default: Date.now }
}, { strict: true }); 

// Create a model
const Customer = mongoose.model('customers', customerSchema);

### Detailed schema creation

```
// Define the customer schema
const customerSchema = new mongoose.Schema({

  // Basic data types
  name: {
    type: String, // String data type for the customer's name
    required: true, // Makes the "name" field mandatory
    trim: true, // Removes leading and trailing whitespace
  },

  // Unique email with validation
  email: {
    type: String,
    required: true,
    unique: true, // Ensures no duplicate emails in the collection
    validate: {
      validator: (value) => /^[^\s@]+@[^\s@]+\.[^\s@]+$/.test(value), // for email validation
      message: 'Invalid email format', // Custom error message for invalid emails
    },
  },

  // Number with minimum and maximum value
  age: {
    type: Number,
    required: true,
    min: 18, // Minimum age limit
    max: 120, // Maximum age limit
  },

  // Date with default value
  createdAt: {
    type: Date,
    default: Date.now, // Sets the creation time automatically
  },

  // Embedded document for address details (optional)
  address: {
    type: {
      street: String, // Street address
      city: String, // City
      state: String, // State/Province
      country: String, // Country
    },
  },

  // Virtual field (not stored in database)
  fullName: {
    type: String,
    virtual: true, // Indicates it's a virtual field
    get() { // Getter function to combine name parts
      return `${this.name.first} ${this.name.last}`; // assuming "name" has "first" and "last"
    },
  },

  // Custom validation using a function
  validateAge: {
    validator: function (value) {
      return value >= 18; // Custom validation for minimum age
    },
    message: 'Age must be 18 or older',
  },

}, { strict: true }); // Enable strict mode for better data validation

```

# Simple database operations

In [5]:
const newCustomer = new Customer({
  name: 'Alice',
  email: 'alice@example.com',
  age: 30
});

// Delete all docs to clear the collection
Customer.deleteMany({})
  .then(result => console.log('Deleted:', result.deletedCount))
  .catch(err => console.error(err));

Promise { <pending> }

Deleted: 0


In [6]:
// Save one document to the database
newCustomer.save()
    .then(customer => console.log('Customer saved:', customer))
    .catch(err => console.error(err));

Promise { <pending> }

Customer saved: {
  name: 'Alice',
  email: 'alice@example.com',
  age: 30,
  _id: new ObjectId('66c99096e512124241efcb58'),
  createdAt: 2024-08-24T07:49:42.165Z,
  __v: 0
}


In [7]:
// Insert many docs
Customer.insertMany([
  { name: 'Bob', email: 'bob@example.com', age: 25 },
  { name: 'Charlie', email: 'charlie@example.com', age: 32 },
  { name: 'David', email: 'david@example.com', age: 40 }
]).then(savedCustomers => console.log('Customers saved:', savedCustomers))
  .catch(err => console.error(err));

Promise { <pending> }

Customers saved: [
  {
    name: 'Bob',
    email: 'bob@example.com',
    age: 25,
    _id: new ObjectId('66c9909ae512124241efcb5b'),
    createdAt: 2024-08-24T07:49:46.828Z,
    __v: 0
  },
  {
    name: 'Charlie',
    email: 'charlie@example.com',
    age: 32,
    _id: new ObjectId('66c9909ae512124241efcb5c'),
    createdAt: 2024-08-24T07:49:46.829Z,
    __v: 0
  },
  {
    name: 'David',
    email: 'david@example.com',
    age: 40,
    _id: new ObjectId('66c9909ae512124241efcb5d'),
    createdAt: 2024-08-24T07:49:46.830Z,
    __v: 0
  }
]


In [8]:
// Find all documents
Customer.find({})
  .then(customers => console.log('Customers:', customers))
  .catch(err => console.error(err));

// Find documents with specific criteria (e.g., age greater than 30)
Customer.find({ age: { $gt: 30 } })
  .then(customers => { 
      // We can use customers.toArray() to load all docs into memory
      console.log('Customers over 30:', customers);
      // customers is a curser: the cursor is iterable, allowing you to process documents one by one without loading them all into memory.
  })
  .catch(err => console.error(err));

// Find a single document by ID (use findById for efficiency)
Customer.findById('64e1234567890abc12345678') // Replace with actual ID
  .then(customer => console.log('Customer:', customer))
  .catch(err => console.error(err));

Promise { <pending> }

Customer: null
Customers: [
  {
    _id: new ObjectId('66c99096e512124241efcb58'),
    name: 'Alice',
    email: 'alice@example.com',
    age: 30,
    createdAt: 2024-08-24T07:49:42.165Z,
    __v: 0
  },
  {
    _id: new ObjectId('66c9909ae512124241efcb5b'),
    name: 'Bob',
    email: 'bob@example.com',
    age: 25,
    createdAt: 2024-08-24T07:49:46.828Z,
    __v: 0
  },
  {
    _id: new ObjectId('66c9909ae512124241efcb5c'),
    name: 'Charlie',
    email: 'charlie@example.com',
    age: 32,
    createdAt: 2024-08-24T07:49:46.829Z,
    __v: 0
  },
  {
    _id: new ObjectId('66c9909ae512124241efcb5d'),
    name: 'David',
    email: 'david@example.com',
    age: 40,
    createdAt: 2024-08-24T07:49:46.830Z,
    __v: 0
  }
]
Customers over 30: [
  {
    _id: new ObjectId('66c9909ae512124241efcb5c'),
    name: 'Charlie',
    email: 'charlie@example.com',
    age: 32,
    createdAt: 2024-08-24T07:49:46.829Z,
    __v: 0
  },
  {
    _id: new ObjectId('66c9909ae512124241efcb5d'),
    name: '

In [9]:
// Update all documents where age is greater than 30
Customer.updateMany({ age: { $gt: 30 } }, { $set: { age: 35 } })
  .then(result => console.log('Updated:', result))
  .catch(err => console.error(err));

Promise { <pending> }

Updated customer: {
  _id: new ObjectId('66c99096e512124241efcb58'),
  name: 'Alice',
  email: 'alice@example.com',
  age: 40,
  createdAt: 2024-08-24T07:49:42.165Z,
  __v: 0
}
Updated: {
  acknowledged: true,
  modifiedCount: 2,
  upsertedId: null,
  upsertedCount: 0,
  matchedCount: 2
}


In [ ]:
// Update the first document that matches the query
/*
In Mongoose, the { new: true } option is used with update operations (like findOneAndUpdate or updateOne)
to specify that you want the updated document to be returned as the result instead of the original document
*/
Customer.findOneAndUpdate({ email: 'alice@example.com' }, { $set: { age: 40 } }, { new: true })
  .then(updatedCustomer => console.log('Updated customer:', updatedCustomer))
  .catch(err => console.error(err));

In [10]:
// Delete all documents where age is greater than 33
Customer.deleteMany({ age: { $gt: 33 } })
  .then(result => console.log('Deleted:', result.deletedCount))
  .catch(err => console.error(err));

Promise { <pending> }

Deleted: 3
Deleted customer: {
  _id: new ObjectId('66c9909ae512124241efcb5b'),
  name: 'Bob',
  email: 'bob@example.com',
  age: 25,
  createdAt: 2024-08-24T07:49:46.828Z,
  __v: 0
}
Deleted: 0


In [ ]:
// Delete the first document that matches the query
Customer.findOneAndDelete({ name: 'Bob' })
  .then(deletedCustomer => console.log('Deleted customer:', deletedCustomer))
  .catch(err => console.error(err));

In [ ]:
// Delete all docs
Customer.deleteMany({})
  .then(result => console.log('Deleted:', result.deletedCount))
  .catch(err => console.error(err));

## Closing the Database Connection

In [11]:
// Closing the connection
mongoose.disconnect()
  .then(() => console.log('Disconnected from MongoDB'))
  .catch(err => console.error(err));

Promise { <pending> }

Disconnected from MongoDB
